In [2]:
import numpy as np
import tensorflow as tf
import os

from skimage.metrics import structural_similarity as ssim   
from PIL import Image
from tensorflow import keras
from keras import layers

In [3]:
#Define the CBR (Convolutional, Batch Normalization and ReLU block)
class CBR(keras.Model):
    
    def __init__(self,filters):
        super().__init__()
        self.Convolution = layers.Conv2D(filters = filters, kernel_size = 3, strides = 1, padding = 'same', use_bias = False)
        self.BatchN = layers.BatchNormalization()
        self.relu = layers.ReLU()

    def call(self,inputs):
        x = self.Convolution(inputs)
        x = self.BatchN(x)
        return self.relu(x)    
    
#Define the Network
inputs = keras.Input(shape = (None,None,3))

CBR1 = CBR(64)(inputs)

CBR2 = CBR(64)(CBR1)

Downsampling1 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR2)
CBR3 = CBR(128)(Downsampling1)

CBR4 = CBR(128)(CBR3)

Downsampling2 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR4)
CBR5 = CBR(256)(Downsampling2)

CBR6 = CBR(256)(CBR5)

Downsampling3 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR6)
CBR7 = CBR(512)(Downsampling3)

Upsampling1 = layers.Conv2DTranspose(filters = 256, kernel_size = 2, strides = 2, padding = 'same')(CBR7)
#Add the Skip connections
combined_images1 = layers.Concatenate(axis = -1)([Upsampling1,CBR6])
CBR8 = CBR(512)(combined_images1)

CBR9 = CBR(256)(CBR8)

Upsampling2 = layers.Conv2DTranspose(filters = 128, kernel_size = 2, strides = 2, padding = 'same')(CBR9)
combined_images2 = layers.Concatenate(axis = -1)([Upsampling2,CBR4])
CBR10 = CBR(256)(combined_images2)

CBR11 = CBR(128)(CBR10)

Upsampling3 = layers.Conv2DTranspose(filters = 64, kernel_size = 2, strides = 2, padding = 'same')(CBR11)
combined_images3 = layers.Concatenate(axis = -1)([Upsampling3,CBR2])
CBR12 = CBR(128)(combined_images3)

CBR13 = CBR(64)(CBR12)

output = layers.Conv2D(filters = 1, kernel_size = 3, strides = 1, padding = 'same')(CBR13)
Mask = keras.Model(inputs = inputs, outputs = output, name = 'Test')

2025-02-13 18:47:19.761933: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
#Load the training data
def load_dataset(top_dir = 'input_data'):
    images_dataset = []
    for name in sorted(os.listdir(top_dir)):
        img = np.array(Image.open(os.path.join(top_dir,name)))
        img = img[:,:,:3]
        images_dataset.append(img)
    return images_dataset

def load_dataset_g(top_dir = 'input_data'):
    images_dataset = []
    for name in sorted(os.listdir(top_dir)):
        img = np.array(Image.open(os.path.join(top_dir,name)))
        images_dataset.append(img)
    return images_dataset

In [5]:
rain_images_a = np.asarray(load_dataset(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/A_Qian_Cam_A+B/TestSet/rain_images'))
rain_images_b = np.asarray(load_dataset(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/B_Quan/TestSet/rain_images'))
# rain_images = np.append(rain_images_a,rain_images_b,axis = 0)
# del(rain_images_a)
# del(rain_images_b)

KwonMask_a = np.asarray(load_dataset_g(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/A_Qian_Cam_A+B/TestSet/KwonMask'))
KwonMask_b = np.asarray(load_dataset_g(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/B_Quan/TestSet/KwonMask'))
# KwonMask = np.append(KwonMask_a,KwonMask_b,axis = 0)
# del(KwonMask_a)
# del(KwonMask_b)

rain_images_a = np.float32(rain_images_a)
rain_images_b = np.float32(rain_images_b)
KwonMask_a = np.float32(KwonMask_a)
KwonMask_b = np.float32(KwonMask_b)

rain_images_a = rain_images_a/255
rain_images_b = rain_images_b/255
KwonMask_a = KwonMask_a/255
KwonMask_b = KwonMask_b/255

In [6]:
Mask.load_weights('/users/lady6758/Mayowa_4YP_Code/ReTraining_ReTesting/Raindrop_Mask/Fold1Retrained/attentionweights.weights.h5')

In [7]:
no_of_images = rain_images_a.shape[0]
s1 = np.empty(shape = no_of_images)
m1 = np.empty(shape = no_of_images)
for i in range(no_of_images):
    img1 = rain_images_a[i]
    img1 = np.expand_dims(img1,axis = 0)
    generated_mask = np.squeeze(np.squeeze(Mask.predict(img1),axis = -1),axis = 0)

    mask = KwonMask_a[i]
    s1[i] = ssim(mask,generated_mask,data_range = 1, gaussian_weights=True, K1 = 0.01, K2 = 0.03, use_sample_covariance = False)
    m1[i] = np.mean((mask - generated_mask)**2)
    print(s1[i])
    print(m1[i])
    print(i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 733ms/step
0.774059210641577
0.008506007492542267
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
0.8639927741433457
0.007476343773305416
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
0.8412369952058436
0.010300985537469387
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
0.6992930055135864
0.0057324632070958614
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
0.7617343177401187
0.005527104716747999
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
0.7303953799412949
0.004699543584138155
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
0.6641823098021341
0.002420445904135704
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
0.6893524142320678
0.002357890596613288
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
0.6853133693189822
0.003989963326603174
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
0.6824394325008636
0.002220127498731017
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
0.7161489806839564
0.0019709693733602762
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
0.7239910721893923
0.0023875152692198753
11
1/1 ━━━━━━━━━━━━━━━━━━━━

In [8]:
no_of_images = rain_images_b.shape[0]
s2 = np.empty(shape = no_of_images)
m2 = np.empty(shape = no_of_images)
for i in range(no_of_images):
    img1 = rain_images_b[i]
    img1 = np.expand_dims(img1,axis = 0)
    generated_mask = np.squeeze(np.squeeze(Mask.predict(img1),axis = -1),axis = 0)

    mask = KwonMask_b[i]
    s2[i] = ssim(mask,generated_mask,data_range = 1, gaussian_weights=True, K1 = 0.01, K2 = 0.03, use_sample_covariance = False)
    m2[i] = np.mean((mask - generated_mask)**2)
    print(s2[i])
    print(m2[i])
    print(i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
0.610605088880106
0.003888441016897559
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
0.5896491963672778
0.00459268968552351
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
0.5541992942765739
0.004575034603476524
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
0.613276551968184
0.0030700762290507555
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
0.5850620663786296
0.003569837659597397
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
0.5706302019721801
0.003219291800633073
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
0.6288461732225641
0.003966605756431818
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
0.6207984672270433
0.005123324226588011
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
0.6471336022119856
0.005581430159509182
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
0.678844176670593
0.0034437438007444143
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
0.6191589750610594
0.0031446265056729317
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
0.6140882174963668
0.0021186647936701775
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

In [9]:
def mean_std(array):
    mean = np.mean(array)
    std = np.std(array)
    print(f'Mean: {mean}')
    print(f'STD: {std}')

In [10]:
testa_mse = mean_std(m1)

Mean: 0.008215849520440195
STD: 0.008647307263862742


In [11]:
testa_ssim = mean_std(s1)

Mean: 0.7317188702891003
STD: 0.11939243625645053


In [12]:
testb_mse = mean_std(m2)

Mean: 0.005294083321812172
STD: 0.0028993514994246124


In [13]:
testb_ssim = mean_std(s2)

Mean: 0.6578949606738781
STD: 0.10932043595655078
